In [258]:
from bs4 import BeautifulSoup
import os
import pandas as pd
!pip install fuzzywuzzy[speedup]
from fuzzywuzzy import process
import re

In [259]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [260]:
directory = '/content/drive/MyDrive/PEK/ФТ'

In [261]:
paragraphs_dataframes = []

In [262]:
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)

    # Check if the file exists and is an HTML file
    if os.path.isfile(file_path) and filename.endswith('.html'):
        # Load the HTML file
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Remove crossed-out text elements
        for tag in soup.find_all(['s', 'strike', 'del']):
            tag.decompose()

        # Extract all paragraphs and concatenate their text
        paragraphs = soup.find_all('p')
        tables = soup.find_all('td')
        spans = soup.find_all('span')
        paragraphs_text = ' '.join(p.get_text(strip=True) for p in paragraphs)
        table_items_text = ' '.join(table.get_text(strip=True) for table in tables)
        span_text = ' '.join(span.get_text(strip=True) for span in spans)
        combined_text = paragraphs_text + ' ' + table_items_text + ' ' + span_text

        # Create a DataFrame for the concatenated paragraphs
        df_paragraphs = pd.DataFrame([[os.path.splitext(filename)[0], combined_text]],
                                     columns=['Query', 'Answer'])

        # Append the DataFrame to the list
        paragraphs_dataframes.append(df_paragraphs)

In [263]:
combined_paragraphs_df = []
combined_paragraphs_df = pd.concat(paragraphs_dataframes, ignore_index=True)

In [264]:
display(combined_paragraphs_df)

,Query,Answer
0,ФТ (EPIC-5293),Функциональные требования Изменить алгоритм ...
1,ФТ (EPIC-6660),Функциональные требования Повысить уровень ...
2,ФТ (EPIC-7260),Функциональные требования Ссылки EPIC-7018-...
3,ФТ (EPIC-7399) Интеграция с интегратором доста...,Функциональные требования Создать справоч...
4,ФТ (EPIC-7065),Функциональные требования 29.06.2021 ДС -д...
...,...,...
194,ФТ (EPIC-13624),Функциональные требования Название системы № э...
195,ФТ (EPIC-13431),Функциональные требования Название системы № э...
196,ФТ (EPIC-13614) Услуга приемки _МЗУ_ и _МЗЯ_,Функциональные требования Название системы № э...
197,ФТ (EPIC-13694),Функциональные требования Название системы № э...


In [265]:
patterns_to_remove = ['…СТАТУС', 'ОЦЕНОЧНАЯ ЛИЦЕНЗИЯВам нравится Confluence? Пожалуйста, рассмотритедля приобретения егосегодня.']
for pattern in patterns_to_remove:
    combined_paragraphs_df['Answer'] = combined_paragraphs_df['Answer'].str.replace(pattern, '', regex=False)

display(combined_paragraphs_df)

,Query,Answer
0,ФТ (EPIC-5293),Функциональные требования Изменить алгоритм ...
1,ФТ (EPIC-6660),Функциональные требования Повысить уровень ...
2,ФТ (EPIC-7260),Функциональные требования Ссылки EPIC-7018-...
3,ФТ (EPIC-7399) Интеграция с интегратором доста...,Функциональные требования Создать справоч...
4,ФТ (EPIC-7065),Функциональные требования 29.06.2021 ДС -д...
...,...,...
194,ФТ (EPIC-13624),Функциональные требования Название системы № э...
195,ФТ (EPIC-13431),Функциональные требования Название системы № э...
196,ФТ (EPIC-13614) Услуга приемки _МЗУ_ и _МЗЯ_,Функциональные требования Название системы № э...
197,ФТ (EPIC-13694),Функциональные требования Название системы № э...


In [267]:
# Check the size of the DataFrame in bytes before removing newline characters
size_before = combined_paragraphs_df.memory_usage(deep=True).sum()
print("Size of DataFrame before removing newlines (in bytes):", size_before)

# Remove all newline characters from the 'Answer' column
combined_paragraphs_df['Answer'] = combined_paragraphs_df['Answer'].str.replace('\n', '')

# Check the size of the DataFrame in bytes after removing newline characters
size_after = combined_paragraphs_df.memory_usage(deep=True).sum()
print("Size of DataFrame after removing newlines (in bytes):", size_after)

Size of DataFrame before removing newlines (in bytes): 35570271
Size of DataFrame after removing newlines (in bytes): 30172358


In [268]:
file_path = '/content/drive/MyDrive/PEK/FT.txt'
encodings = ['cp1251', 'koi8_r', 'iso-8859-5']
for encoding in encodings:
    try:
        links= pd.read_csv(file_path, delimiter=';', header=None, names=['File', 'Link'], encoding=encoding)
        print(f"Successfully loaded with encoding: {encoding}")
        break
    except Exception as e:
        print(f"Error with encoding {encoding}: {e}")

display(links) #links

Successfully loaded with encoding: cp1251


,File,Link
0,Копия ФТ (EPIC-11361),http://confluence/pages/viewpage.action?pageId...
1,НЕ АКТУАЛЬНОЕ! ФТ (EPIC-10992) Выбор продукта ...,http://confluence/pages/viewpage.action?pageId...
2,ФТ (EPIC 5844) Интеграция с проценщиком посред...,http://confluence/pages/viewpage.action?pageId...
3,ФТ (EPIC-10132),http://confluence/pages/viewpage.action?pageId...
4,ФТ (EPIC-10142),http://confluence/pages/viewpage.action?pageId...
...,...,...
823,ФТ Реализация алгоритма расчета по новой услуг...,http://confluence/pages/viewpage.action?pageId...
824,ФТ Создание СУ Скидка на услугу Организация ст...,http://confluence/pages/viewpage.action?pageId...
825,"ФТ СУ Консолидированный забор для компаний, вх...",http://confluence/pages/viewpage.action?pageId...
826,ФТ_2 (EPIC-11408),http://confluence/pages/viewpage.action?pageId...


In [269]:
# Function to extract numeric part
def extract_number(text):
    match = re.search(r'\d+', text)
    return match.group(0) if match else None

In [270]:
# Extract numbers from Query and File columns
combined_paragraphs_df['Key'] = combined_paragraphs_df['Query'].apply(extract_number)
links['Key'] = links['File'].apply(extract_number)

In [271]:
# Merge DataFrames on the extracted numeric key
merged_ft_df = pd.merge(combined_paragraphs_df, links, on='Key', how='left')

In [272]:
# Drop the Key column as it's no longer needed
merged_ft_df.drop(columns=['Key'], inplace=True)

In [273]:
# Drop duplicates based on the 'Query' column
merged_ft_df = merged_ft_df.drop_duplicates(subset=['Query'])

In [278]:
print(merged_ft_df.loc[183])

Query                        ФТ на механизм выгрузки из CDC
Answer    Функциональные требования 19.02.2024    Оптиум...
File                ФТ дополнение к "Оплата по СБП на ТВИО"
Link      http://confluence/pages/viewpage.action?pageId...
Name: 183, dtype: object


In [279]:
merged_ft_df.at[183, 'Link'] = 'http://confluence/pages/viewpage.action?pageId=176377239&src=contextnavpagetreemode'

In [280]:
print(merged_ft_df.loc[183])

Query                        ФТ на механизм выгрузки из CDC
Answer    Функциональные требования 19.02.2024    Оптиум...
File                ФТ дополнение к "Оплата по СБП на ТВИО"
Link      http://confluence/pages/viewpage.action?pageId...
Name: 183, dtype: object


In [282]:
merged_ft_df = merged_ft_df.drop(columns=['File'])

In [283]:
merged_ft_df['Department'] = 'FT'

In [284]:
merged_ft_df.to_csv('/content/drive/MyDrive/merged_ft_df.csv', index=False)